<a href="https://colab.research.google.com/github/peremartra/Tailoring-LLM-Architectures/blob/main/CH06/CH06_NB02_Logits_KLD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tailoring LLM Architectures**
## **Chapter 6: Knowledge Distillation with Logits Only**
### Training a pruned model using only Logits and KL Divergence

by [Pere Martra](https://github.com/peremartra)

[![LinkedIn](https://img.shields.io/badge/LinkedIn-0077B5?style=flat&logo=linkedin&logoColor=white)](https://www.linkedin.com/in/pere-martra/) [![GitHub](https://img.shields.io/badge/GitHub-100000?style=flat&logo=github&logoColor=white)](https://github.com/peremartra) [![Hugging Face](https://img.shields.io/badge/🤗%20Hugging%20Face-blue)](https://huggingface.co/oopere)

---

**Hardware Environment:** NVIDIA A100 GPU
- **Model:** google/gemma-3-270m (Teacher: 18 transformer blocks) → Pruned Student (14 transformer blocks)
- **Dataset:** Cosmopedia (40,000 samples, 3 epochs)

---

**What we'll accomplish:**
- Train a depth-pruned student model using **only logits and KL Divergence**
- No hidden state alignment (simpler, faster training)
- Use the **Selected-Layer mapping strategy** (map to original indices before pruning)
- Save the trained model to Hugging Face as **gem-3-small**

## Section 0: Environment & Dependencies

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
print("✓ Drive mounted")

In [ ]:
#Samples in the training dataset.
RECOVERY_SAMPLES = 40000

EPOCHS=3
LEARNING_RATE=4e-5
BATCH_SIZE = 16

# Define benchmark tasks
BATCH_EVAL="auto"
RUN_FULL_BENCHMARKS = True
BENCHMARK_LIMIT = None
BENCHMARK_TASKS = [
      "arc_easy",
      "winogrande",
      "hellaswag",
      "lambada_openai",
      "piqa"
]

# Hugging Face Model Name
HF_MODEL_NAME = "gem-3-small"

In [ ]:
# Install required packages
!pip install -q transformers accelerate datasets
!pip install -q optipfair  # For creating pruned model on-the-fly
!pip install -q matplotlib seaborn tqdm
!pip install -q lm_eval
!pip install -q langdetect
!pip install -q codecarbon
!pip install -q huggingface_hub

In [ ]:
import torch, gc
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer
)
from datasets import load_dataset
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tqdm.auto import tqdm
from copy import deepcopy
import warnings
import time

warnings.filterwarnings('ignore')

# Check PyTorch version and device
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
def set_seed(seed=42):
    """Set random seed for reproducibility"""
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

set_seed(42)
print("✓ Random seed set to 42")

In [ ]:
# Download utils.py from GitHub repository
!wget -q https://raw.githubusercontent.com/peremartra/Rearchitecting-LLMs/main/utils.py

# Verify download
import os
if os.path.exists('utils.py'):
    print("✅ utils.py downloaded successfully")
else:
    print("❌ Failed to download utils.py")

from utils import (
    evaluate_metrics,  # Loss & Perplexity
    clear_gpu_cache
)

## Section 1: Load Models and Create Pruned Student

In [ ]:
MODEL_NAME = "google/gemma-3-270m"

print(f"Loading Teacher model: {MODEL_NAME}")
teacher_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
    device_map="auto" if torch.cuda.is_available() else None
)

# Freeze teacher (we never update it)
teacher_model.eval()
for param in teacher_model.parameters():
    param.requires_grad = False

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Print architecture info
n_teacher_layers = len(teacher_model.model.layers)
hidden_dim = teacher_model.config.hidden_size

print(f"\n{'='*60}")
print(f"Teacher Model: {MODEL_NAME}")
print(f"{'='*60}")
print(f"Total layers: {n_teacher_layers}")
print(f"Hidden dimension: {hidden_dim}")
print(f"Total parameters: {teacher_model.num_parameters():,}")
print(f"Memory footprint: {teacher_model.get_memory_footprint() / 1e9:.2f} GB")
print(f"{'='*60}\n")

## Section 2: Prepare Training Dataset

In [ ]:
# ==============================================================================
# Section 2: Prepare Training Dataset (COMPLETE VERSION)
# ==============================================================================

from datasets import load_dataset, Dataset
from tqdm.auto import tqdm
import torch
from torch.utils.data import TensorDataset, DataLoader, random_split

# Define tokenization parameters
MAX_LENGTH = 512

def tokenize_function(examples):
    """Tokenize text samples for training"""
    texts = examples['text'] if isinstance(examples, dict) else examples
    return tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=MAX_LENGTH,
        return_tensors="pt"
    )

print(f"✓ Tokenization function ready (max_length={MAX_LENGTH})")

# Load Cosmopedia dataset
print("\nLoading Cosmopedia dataset...")
dataset_name = "HuggingFaceTB/cosmopedia"
subsets = ["stories", "wikihow", "openstax", "web_samples_v1"]
samples_per_subset = int(RECOVERY_SAMPLES / 4)
num_samples = samples_per_subset * len(subsets)

print(f"Loading {len(subsets)} subsets with {samples_per_subset:,} samples each...")

all_samples = []
for subset in subsets:
    print(f"  Loading {subset}...")
    subset_data = load_dataset(dataset_name, subset, split="train", streaming=True)
    subset_samples = list(subset_data.take(samples_per_subset))
    all_samples.extend(subset_samples)
    print(f"    ✓ {len(subset_samples):,} samples from {subset}")

print(f"✓ Total samples loaded: {len(all_samples):,}")

# Convert to HuggingFace Dataset
distillation_dataset = Dataset.from_dict({'text': [s['text'] for s in all_samples]})
print(f"✓ Cosmopedia dataset ready: {len(distillation_dataset):,} samples")

# ==============================================================================
# Tokenization and Train/Val Split
# ==============================================================================

print("\nPreparing DataLoader with Train/Val split...")

# 1. Tokenize all samples
print("  Converting dataset to list...")
dataset_list = list(distillation_dataset)
texts = [item['text'] for item in dataset_list]

print(f"  Tokenizing {len(texts):,} samples...")
tokenized_data = []
batch_size = 1000
for i in tqdm(range(0, len(texts), batch_size), desc="Tokenizing"):
    batch_texts = texts[i:i+batch_size]
    batch_tokens = tokenizer(
        batch_texts,
        truncation=True,
        padding="max_length",
        max_length=MAX_LENGTH,
        return_tensors="pt"
    )
    tokenized_data.append(batch_tokens)

# Combine all batches
input_ids = torch.cat([batch['input_ids'] for batch in tokenized_data], dim=0)
attention_mask = torch.cat([batch['attention_mask'] for batch in tokenized_data], dim=0)

# 2. Create full dataset
full_dataset = TensorDataset(input_ids, attention_mask)

# 3. Split into Train (80%) and Validation (20%)
generator = torch.Generator().manual_seed(42)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size

train_dataset, val_dataset = random_split(
    full_dataset,
    [train_size, val_size],
    generator=generator
)

# 4. Create DataLoaders
train_dataloader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True
)

eval_dataloader_raw = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False
)

print(f"\n✓ DataLoaders ready:")
print(f"  Total samples: {len(full_dataset):,}")
print(f"  - Training samples: {len(train_dataset):,} (80%) ({len(train_dataloader):,} batches)")
print(f"  - Validation samples: {len(val_dataset):,} (20%) ({len(eval_dataloader_raw):,} batches)")
print(f"  Batch size: {BATCH_SIZE}")

# ==============================================================================
# Evaluation DataLoader Wrapper
# ==============================================================================

class DictDataLoader:
    """Wrapper to convert TensorDataset tuples to dictionaries"""
    def __init__(self, dataloader):
        self.dataloader = dataloader

    def __iter__(self):
        for input_ids, attention_mask in self.dataloader:
            yield {
                'input_ids': input_ids,
                'attention_mask': attention_mask
            }

    def __len__(self):
        return len(self.dataloader)

eval_dataloader = DictDataLoader(eval_dataloader_raw)
print("✓ Evaluation dataloader ready (using validation split)")

## Section 3: Create Pruned Model

In [ ]:
import optipfair as opf

print("Creating Student model with depth pruning...")

# IMPORTANT: Use deepcopy to avoid modifying the original model
student_model = deepcopy(teacher_model)
importance_scores = opf.analyze_layer_importance(student_model, train_dataloader, show_progress=True)

In [ ]:
LAYERS_TO_REMOVE = sorted(importance_scores.keys(), key=lambda x: importance_scores[x])[:4]
print(f"Layers to remove (least important): {LAYERS_TO_REMOVE}")

In [ ]:
student_model = opf.prune_model_depth(
    model=student_model,
    layer_indices=LAYERS_TO_REMOVE,
    show_progress=True,
)

# IMPORTANT: Unfreeze student parameters!
for param in student_model.parameters():
    param.requires_grad = True

# Get student info
n_student_layers = len(student_model.model.layers)

print(f"\n{'='*60}")
print(f"Student Model (Depth Pruned)")
print(f"{'='*60}")
print(f"Total layers: {n_student_layers} (removed {LAYERS_TO_REMOVE})")
print(f"Hidden dimension: {hidden_dim} (unchanged)")
print(f"Total parameters: {student_model.num_parameters():,}")
print(f"{'='*60}\n")

print(f"✓ Student has {n_student_layers} layers vs Teacher's {n_teacher_layers} layers")
print(f"✓ Student parameters unfrozen and ready for training")

In [ ]:
# Evaluate Teacher (baseline)
print("Evaluating Teacher...")
teacher_metrics = evaluate_metrics(teacher_model, eval_dataloader, device=device)
teacher_loss = teacher_metrics['loss']
teacher_ppl = teacher_metrics['perplexity']

if RUN_FULL_BENCHMARKS:
  from utils import model_evaluation
  benchmark_results = {}
  results = model_evaluation(
      model_obj=teacher_model,
      tokenizer=tokenizer,
      tasks=BENCHMARK_TASKS,
      device=device,
      limit=BENCHMARK_LIMIT,
      batch_size=BATCH_EVAL
      )
  benchmark_results['teacher'] = results

In [ ]:
# Evaluate Pruned Student (no training)
print("Evaluating Pruned Student (no training)...")
student_pruned = deepcopy(student_model)
student_metrics = evaluate_metrics(student_pruned, eval_dataloader, device=device)
student_loss = student_metrics['loss']
student_ppl = student_metrics['perplexity']

if RUN_FULL_BENCHMARKS:
    results = model_evaluation(
    model_obj=student_pruned,
    tokenizer=tokenizer,
    tasks=BENCHMARK_TASKS,
    device=device,
    limit=BENCHMARK_LIMIT,
    batch_size=BATCH_EVAL
    )

    benchmark_results['student_pruned'] = results

del student_pruned
clear_gpu_cache()

## Section 4: Implement Compound Loss (Logits Only)

In [ ]:
def compute_logits_only_loss(
    student_logits,
    teacher_logits,
    labels,
    alpha=0.5,       # weight for task loss (Cross-Entropy)
    beta=0.5,        # weight for logits loss (KL Divergence)
    temperature=2.0
):
    """
    Compound loss using ONLY logits (no hidden states).
    
    Components:
    1. Task Loss: Standard cross-entropy with hard labels
    2. Logits Loss: KL Divergence between teacher and student soft labels
    """
    # 1. TASK LOSS (Cross-Entropy)
    shift_logits = student_logits[..., :-1, :].contiguous()
    shift_labels = labels[..., 1:].contiguous()
    loss_task = F.cross_entropy(
        shift_logits.view(-1, shift_logits.size(-1)),
        shift_labels.view(-1),
        ignore_index=-100
    )

    # 2. LOGITS LOSS (KL Divergence)
    student_soft = F.log_softmax(student_logits / temperature, dim=-1)
    teacher_soft = F.softmax(teacher_logits / temperature, dim=-1)

    student_soft = student_soft[..., :-1, :].contiguous()
    teacher_soft = teacher_soft[..., :-1, :].contiguous()

    loss_logits = F.kl_div(
        student_soft.view(-1, student_soft.size(-1)),
        teacher_soft.view(-1, teacher_soft.size(-1)),
        reduction='batchmean'
    ) * (temperature ** 2)

    # COMBINE LOSSES
    total_loss = alpha * loss_task + beta * loss_logits

    loss_dict = {
        'total': total_loss.item(),
        'task': loss_task.item(),
        'logits': loss_logits.item()
    }

    return total_loss, loss_dict

## Section 5: Training Loop (Logits Only)

In [ ]:
def train_student_logits_only(
    student_model,
    teacher_model,
    dataloader,
    alpha=0.5,
    beta=0.5,
    temperature=2.0,
    epochs=3,
    learning_rate=4e-5,
    experiment_name="experiment",
    accumulation_steps=4
):
    """
    Train student model using ONLY logits (no hidden states).
    This is faster and simpler than full knowledge distillation.
    """
    optimizer = torch.optim.AdamW(student_model.parameters(), lr=learning_rate)

    student_model.train()
    teacher_model.eval()

    loss_history = {'total': [], 'task': [], 'logits': []}
    epoch_times = []

    print(f"\n{'='*60}")
    print(f"Starting Training: {experiment_name}")
    print(f"{'='*60}")
    print(f"Epochs: {epochs}")
    print(f"Learning rate: {learning_rate}")
    print(f"Loss weights: α={alpha} (task), β={beta} (logits)")
    print(f"Temperature: {temperature}")
    print(f"Hidden states computation: DISABLED (Logits Only)")
    print(f"Gradient Accumulation Steps: {accumulation_steps}")
    print(f"Effective Batch Size: {dataloader.batch_size * accumulation_steps}")
    print(f"{'='*60}\n")

    total_start_time = time.time()

    for epoch in range(epochs):
        epoch_start_time = time.time()

        epoch_losses = {k: [] for k in loss_history.keys()}
        progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}")

        accumulated_losses = {k: 0.0 for k in loss_history.keys()}
        accumulation_counter = 0

        for batch_idx, (input_ids, attention_mask) in enumerate(progress_bar):
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = input_ids.clone()

            # Student forward pass (NO hidden states)
            student_outputs = student_model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                output_hidden_states=False  # KEY: No hidden states
            )

            # Teacher forward pass (no gradients, NO hidden states)
            with torch.no_grad():
                teacher_outputs = teacher_model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    output_hidden_states=False  # KEY: No hidden states
                )

            # Compute logits-only loss
            loss, loss_dict = compute_logits_only_loss(
                student_logits=student_outputs.logits,
                teacher_logits=teacher_outputs.logits,
                labels=labels,
                alpha=alpha,
                beta=beta,
                temperature=temperature
            )

            scaled_loss = loss / accumulation_steps
            scaled_loss.backward()

            for key in accumulated_losses:
                accumulated_losses[key] += loss_dict[key]
            accumulation_counter += 1

            if (batch_idx + 1) % accumulation_steps == 0:
                torch.nn.utils.clip_grad_norm_(student_model.parameters(), max_norm=1.0)
                optimizer.step()
                optimizer.zero_grad()

                avg_losses = {k: v / accumulation_counter for k, v in accumulated_losses.items()}
                for key in avg_losses:
                    epoch_losses[key].append(avg_losses[key])

                progress_bar.set_postfix({
                    'loss': f"{avg_losses['total']:.4f}",
                    'task': f"{avg_losses['task']:.4f}",
                    'logits': f"{avg_losses['logits']:.4f}"
                })

                accumulated_losses = {k: 0.0 for k in loss_history.keys()}
                accumulation_counter = 0

        if accumulation_counter > 0:
            torch.nn.utils.clip_grad_norm_(student_model.parameters(), max_norm=1.0)
            optimizer.step()
            optimizer.zero_grad()

            avg_losses = {k: v / accumulation_counter for k, v in accumulated_losses.items()}
            for key in avg_losses:
                epoch_losses[key].append(avg_losses[key])

        for key in epoch_losses:
            if epoch_losses[key]:
                avg_loss = np.mean(epoch_losses[key])
                loss_history[key].append(avg_loss)

        epoch_time = time.time() - epoch_start_time
        epoch_times.append(epoch_time)

        print(f"Epoch {epoch+1} avg losses - "
              f"Total: {loss_history['total'][-1]:.4f}, "
              f"Task: {loss_history['task'][-1]:.4f}, "
              f"Logits: {loss_history['logits'][-1]:.4f} "
              f"[{epoch_time:.1f}s]")

    total_time = time.time() - total_start_time

    loss_history['epoch_times_seconds'] = epoch_times
    loss_history['total_time_seconds'] = total_time

    print(f"\n✓ Training completed: {experiment_name}")
    print(f"  Total time: {total_time:.1f}s ({total_time/60:.1f} min)")
    print(f"  Avg time per epoch: {np.mean(epoch_times):.1f}s")

    return student_model, loss_history

## Section 6: Train with Logits Only

In [ ]:
# Create fresh student model
student_logits_only = deepcopy(student_model)

# Train with logits-only
student_trained, history = train_student_logits_only(
    student_model=student_logits_only,
    teacher_model=teacher_model,
    dataloader=train_dataloader,
    alpha=0.5,
    beta=0.5,
    temperature=2.0,
    epochs=EPOCHS,
    learning_rate=LEARNING_RATE,
    experiment_name="Logits-Only KLD Training"
)

In [ ]:
print("Evaluating Trained Student (Logits Only)...")
trained_metrics = evaluate_metrics(student_trained, eval_dataloader, device=device)
trained_loss = trained_metrics['loss']
trained_ppl = trained_metrics['perplexity']

if RUN_FULL_BENCHMARKS:
    results = model_evaluation(
        model_obj=student_trained,
        tokenizer=tokenizer,
        tasks=BENCHMARK_TASKS,
        device=device,
        limit=BENCHMARK_LIMIT,
        batch_size=BATCH_EVAL
    )

    benchmark_results['logits_only_trained'] = results

## Section 7: Results Summary

In [ ]:
print("\n" + "="*60)
print("RESULTS SUMMARY")
print("="*60)
print(f"\nPerplexity Comparison:")
print(f"  Teacher (baseline):     {teacher_ppl:.2f}")
print(f"  Student (pruned, no KD): {student_ppl:.2f}")
print(f"  Student (Logits-Only KD): {trained_ppl:.2f}")

# Calculate recovery
degradation = student_ppl - teacher_ppl
recovered = student_ppl - trained_ppl
recovery_pct = (recovered / degradation) * 100 if degradation > 0 else 0

print(f"\nRecovery Analysis:")
print(f"  Degradation from pruning: +{degradation:.2f} PPL")
print(f"  Recovered via Logits KD:  -{recovered:.2f} PPL")
print(f"  Recovery percentage:       {recovery_pct:.1f}%")
print("="*60)

In [ ]:
# Plot training curves
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for idx, (key, title) in enumerate([('total', 'Total Loss'), ('task', 'Task Loss (CE)'), ('logits', 'Logits Loss (KLD)')]):
    axes[idx].plot(history[key], marker='o')
    axes[idx].set_title(title)
    axes[idx].set_xlabel('Epoch')
    axes[idx].set_ylabel('Loss')
    axes[idx].grid(True, alpha=0.3)

plt.suptitle('Training Progress: Logits-Only Knowledge Distillation', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## Section 8: Save Model to Hugging Face

In [ ]:
from huggingface_hub import login, HfApi

# Login to Hugging Face
# You'll need to enter your HF token when prompted
login()

In [ ]:
# Save the trained model to Hugging Face
print(f"\nSaving model to Hugging Face as '{HF_MODEL_NAME}'...")

# Push model to hub
student_trained.push_to_hub(
    HF_MODEL_NAME,
    commit_message="Depth-pruned Gemma-3-270m trained with Logits-Only KD"
)

# Push tokenizer to hub
tokenizer.push_to_hub(
    HF_MODEL_NAME,
    commit_message="Tokenizer for gem-3-small"
)

print(f"\n✓ Model saved successfully!")
print(f"  Model URL: https://huggingface.co/{HF_MODEL_NAME}")

In [ ]:
# Create model card
model_card_content = f"""
---
license: apache-2.0
language:
- en
base_model: google/gemma-3-270m
tags:
- knowledge-distillation
- depth-pruning
- gemma
---

# {HF_MODEL_NAME}

## Model Description

This is a depth-pruned version of google/gemma-3-270m, trained using Knowledge Distillation with Logits-Only KL Divergence.

### Architecture
- **Base Model**: google/gemma-3-270m (18 transformer blocks)
- **Pruned Model**: 14 transformer blocks (4 least important blocks removed)
- **Parameters**: ~{student_trained.num_parameters():,}

### Training Details
- **Dataset**: Cosmopedia (40,000 samples)
- **Method**: Knowledge Distillation with KL Divergence on logits
- **Epochs**: {EPOCHS}
- **Learning Rate**: {LEARNING_RATE}
- **Loss Weights**: α={0.5} (task), β={0.5} (logits)
- **Temperature**: 2.0

### Performance
- **Teacher Perplexity**: {teacher_ppl:.2f}
- **Pruned (no KD) Perplexity**: {student_ppl:.2f}
- **Trained Perplexity**: {trained_ppl:.2f}
- **Recovery**: {recovery_pct:.1f}%

## Usage

```python
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("{HF_MODEL_NAME}")
tokenizer = AutoTokenizer.from_pretrained("{HF_MODEL_NAME}")

# Generate text
input_text = "Once upon a time"
inputs = tokenizer(input_text, return_tensors="pt")
outputs = model.generate(**inputs, max_length=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
```

## Citation

If you use this model, please cite:

```bibtex
@misc{{
  author = {{Pere Martra}},
  title = {{gem-3-small: Depth-pruned Gemma with Knowledge Distillation}},
  year = {{2026}},
  publisher = {{Hugging Face}},
  url = {{https://huggingface.co/{HF_MODEL_NAME}}}
}}
```
"""

# Save model card
api = HfApi()
api.upload_file(
    path_or_fileobj=model_card_content.encode(),
    path_in_repo="README.md",
    repo_id=HF_MODEL_NAME,
    repo_type="model"
)

print("✓ Model card created and uploaded!")

## Cleanup

In [ ]:
# Cleanup to free GPU memory
del student_logits_only
clear_gpu_cache()

print("\n" + "="*60)
print("TRAINING COMPLETE")
print("="*60)
print(f"\nThe trained model has been saved to Hugging Face as '{HF_MODEL_NAME}'")
print(f"URL: https://huggingface.co/{HF_MODEL_NAME}")
print("\nYou can load it using:")
print(f"  model = AutoModelForCausalLM.from_pretrained('{HF_MODEL_NAME}')")
print("="*60)